In [12]:
import sys, random, os
import numpy as np
from matplotlib import pyplot as plt
import pydot
import cv2
import util
import midi


import keras
print("Keras Version: " + keras.__version__)
from keras.layers import Input, Dense, Activation, Dropout, Flatten, Reshape, Permute, RepeatVector, ActivityRegularization, TimeDistributed, Lambda, SpatialDropout1D
from keras.layers.convolutional import Conv1D, Conv2D, Conv2DTranspose, UpSampling2D, ZeroPadding2D
from keras.layers.embeddings import Embedding
from keras.layers.local import LocallyConnected2D
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.noise import GaussianNoise
from tensorflow.keras.layers import BatchNormalization
from keras.layers.recurrent import LSTM, SimpleRNN
from keras.initializers import RandomNormal
from keras.losses import binary_crossentropy
from keras.models import Model, Sequential, load_model
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras.utils.vis_utils import plot_model
from keras import backend as K
from keras import regularizers
from tensorflow.keras.layers import Layer, InputSpec

Keras Version: 2.7.0


In [ ]:
x_in = Input(shape=x_shape[1:])
x = Embedding(x_train.shape[0], PARAM_SIZE, input_length=1)(x_in)
x = Flatten(name='pre_encoder')(x)

z_mean = Dense(PARAM_SIZE)(x)
z_log_sigma_sq = Dense(PARAM_SIZE)(x)
x = Lambda(vae_sampling, output_shape=(PARAM_SIZE,), name='pre_encoder')([z_mean, z_log_sigma_sq])

In [ ]:
x = Dense(1600, name='encoder')(x)
x = BatchNormalization(momentum=BN_M)(x)
x = Activation('relu')(x)
x = Dropout(DO_RATE)(x)

x = Dense(MAX_LENGTH * 200)(x)
print K.int_shape(x)
x = Reshape((MAX_LENGTH, 200))(x)
x = TimeDistributed(BatchNormalization(momentum=BN_M))(x)
x = Activation('relu')(x)
x = Dropout(DO_RATE)(x)

x = TimeDistributed(Dense(2000))(x)
x = TimeDistributed(BatchNormalization(momentum=BN_M))(x)
x = Activation('relu')(x)
x = Dropout(DO_RATE)(x)
print K.int_shape(x)

x = TimeDistributed(Dense(y_shape[2] * y_shape[3], activation='sigmoid'))(x)
x = Reshape((y_shape[1], y_shape[2], y_shape[3]))(x)

model = Model(x_in, x)
model.compile(optimizer=Adam(lr=LR), loss=vae_loss)